<a href="https://colab.research.google.com/github/karengarm/NLP_Disaster_Tweets_Classification/blob/main/03_BERT_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classification of Tweets During Crisis-Events using the BERT model

This notebook uses the BERT model to classify tweets as informative of non-informative. For a more detailed description of the problem, the data set, the preprocessing steps and for a basline approach, please see the "Automatic detection of crisis-related messages - NLP.ipynb" notebook.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/My\ Drive/CS230

In [ ]:
!pip install emoji
!pip install langdetect
!pip install tensorflow_text

In [ ]:
import pandas as pd
import numpy as np
import emoji
from google_trans_new_local import google_translator
from langdetect import detect
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, roc_curve, auc
import tensorflow as tf
from sklearn.model_selection import train_test_split
import tensorflow_hub as hub
import tensorflow_text as text
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.utils.fixes import loguniform
from sklearn.model_selection import ParameterSampler


from numpy.random import seed
seed(1)
from tensorflow.random import set_seed
set_seed(2)

## Scenario 1
How well can a model trained on one type of disaster (i.e floods, shootings, etc.) perform on similar events of the same type?

In [ ]:
def get_data(category):
  df_train = pd.read_csv('train_data.csv', header = 0, sep = 't')
  df_train = df_train.dropna(subset=['tweettext_proc'])
  df_test = pd.read_csv('test_data.csv', header = 0, sep = 't')
  df_test = df_test.dropna(subset=['tweettext_proc'])

  labels_dict = {'Not informative': 0, 'Related and informative':1}
  df_train['labels'] = df_train.cat_informativeness.replace(labels_dict)
  df_test['labels'] = df_test.cat_informativeness.replace(labels_dict)

  train = df_train.loc[df_train['categorization_type'] == category]
  test = df_test.loc[df_test['categorization_type'] == category]

  X_train = train['tweettext_proc']
  y_train = tf.keras.utils.to_categorical(train['labels'], num_classes=2)

  X_test = test['tweettext_proc']
  y_test = tf.keras.utils.to_categorical(test['labels'], num_classes=2)


  return X_train, y_train, X_test, y_test

### Defining the model
The BERT model consist of a pre-trained BERT endocing layer, a drop-out layer for regularization and a dense layer using a sigmoid activation function. The model was partly based on [this model](https://www.kaggle.com/code/sanketsonu/bert-model-nlp-with-disaster-tweets) posted on kaggle by Sanket Sonu.

The optimizer used is the Adam optimizer, which is used together with the binary crossentropy loss since it is a binary classification problem.

In [ ]:
def create_bert(learning_rate=1e-3, dropout=0.1):
    # Input layer
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='Text')

    # Bert encoding
    bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3", name = "Bert_preprocessing")
    bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_cased_L-24_H-1024_A-16/4", name = "Bert_encoding")
    preprocessed_text = bert_preprocess(text_input)
    outputs = bert_encoder(preprocessed_text)

    # Classification layer with dropout for regularization
    l = tf.keras.layers.Dropout(dropout, name='Dropout')(outputs['pooled_output'])
    l = tf.keras.layers.Dense(2, activation='sigmoid', name='Classifier')(l)

    # Compile model
    model = tf.keras.Model(inputs=[text_input], outputs=[l])
    model.compile(optimizer= tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='binary_crossentropy',
                  metrics=tf.keras.metrics.CategoricalAccuracy(name='accuracy'))


    return model

### Evaluation

Evaluating the model using accuracy, precision, recall, f1-score and mauc.

In [ ]:
def evaluate(y_test, y_pred, trhold):
    """
    Evaluation function. For each of the text in evaluation data, it reads the score from
    the predictions made. And based on this, it calculates the values of
    True positive, True negative, False positive, and False negative.

    :param y_test: true labels
    :param y_pred: predicted labels
    :return: evaluation metrics for classification: accuracy, precision, recall, f1_score and mauc
    """
    # calculating mauc
    fpr, tpr, thresholds = roc_curve(y_test, y_pred, pos_label=1)
    mauc = auc(fpr, tpr)

    y_pred  = np.where(y_pred > trhold, 1, 0) 
    labels = ['Not informative', 'Informative']

    # calculating accuracy_score, precision_score, recall_score, f1_score
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1score = f1_score(y_test, y_pred)
    
    
    report = classification_report(y_test, y_pred, target_names = labels)
    print(report)
    print('mauc: ', mauc)
    return accuracy, precision, recall, f1score, mauc 

### Running the model
In scenario 1 the model is trained using data from one of three categories and tested on data from the same category.

In [ ]:
def run_scenario1(learning_rate=1e-3, dropout=0.1, epochs=5, batch_size=16):  
  for cat in ['Earthquake', 'Floods', 'Derailment']:

      # Get data
      X_train, y_train, X_test, y_test = get_data(cat)

      #Training
      bert_model = create_bert(learning_rate, dropout)
      hist = bert_model.fit(x=X_train, y=y_train, epochs=epochs, batch_size=batch_size, validation_split=.15)

      #Testing
      y_test_arg = np.argmax(y_test, axis=1)
      y_pred = np.argmax(bert_model.predict(X_test),axis=1)

      print(cat)
      accuracy, precision, recall, f1score, mauc = evaluate(y_test_arg, y_pred, 0.5)

In [ ]:
run_scenario1()

Epoch 1/5
184/184 [==============================] - 119s 550ms/step - loss: 0.7092 - accuracy: 0.5744 - val_loss: 0.7184 - val_accuracy: 0.5269
Epoch 2/5
184/184 [==============================] - 100s 543ms/step - loss: 0.6627 - accuracy: 0.6196 - val_loss: 0.6341 - val_accuracy: 0.6385
Epoch 3/5
184/184 [==============================] - 100s 544ms/step - loss: 0.6277 - accuracy: 0.6519 - val_loss: 0.6816 - val_accuracy: 0.5673
Epoch 4/5
184/184 [==============================] - 100s 543ms/step - loss: 0.6143 - accuracy: 0.6730 - val_loss: 0.5995 - val_accuracy: 0.6846
Epoch 5/5
32/32 [==============================] - 31s 928ms/step
Earthquake
                 precision    recall  f1-score   support

Not informative       0.51      0.66      0.58       373
    Informative       0.76      0.63      0.69       627

       accuracy                           0.64      1000
      macro avg       0.64      0.65      0.63      1000
   weighted avg       0.67      0.64      0.65      1000

## Hyperparameter tuning for scenario 1

The best hyperparameters are found by randomly choosing a combination of values for the following hyperparameters:

* Learning Rate using log scale
* Number of epochs
* Batch size
* Dropout

In [ ]:
rng = np.random.RandomState(2)
param_dist = {    'dropout': [0.1, 0.3],
                  'learning_rate': loguniform.rvs(1e-5, 1e-2, size= 10),
                  'epochs': [2, 5, 10, 25],
                  'batch_size': [16, 32, 64, 128]
                  }
dict_parameters = ParameterSampler(param_distributions=param_dist, n_iter=10, random_state=rng)

In [ ]:
for parameters in dict_parameters:
    print("===============Now testing the following parameters===============")
    print("Learning rate: ", parameters['learning_rate'])
    print("Dropout: ", parameters['dropout'])
    print("Epochs: ", parameters['epochs'])
    print("Batch size: ", parameters['batch_size'])
    print()

    run_scenario1(learning_rate=parameters['learning_rate'], dropout=parameters['dropout'], epochs=parameters['epochs'], batch_size=parameters['batch_size'])


===============Now testing the following parameters===============
Learning rate:  5.659770986133262e-05
Dropout:  0.1
Epochs:  25
Batch size:  16
Epoch 1/25
184/184 [==============================] - 118s 547ms/step - loss: 0.7216 - accuracy: 0.5496 - val_loss: 0.7215 - val_accuracy: 0.4577
Epoch 2/25
184/184 [==============================] - 102s 553ms/step - loss: 0.6935 - accuracy: 0.5795 - val_loss: 0.6820 - val_accuracy: 0.5596
Epoch 3/25
184/184 [==============================] - 107s 583ms/step - loss: 0.6870 - accuracy: 0.5795 - val_loss: 0.6934 - val_accuracy: 0.5288
Epoch 4/25
 55/184 [=======>......................] - ETA: 1:00 - loss: 0.6946 - accuracy: 0.5864

## Scenario 2

How well can a model trained on many type of disaster (i.e floods, shootings, etc.) perform on one disaster type?

In [ ]:
def get_train_data():
  df_train = pd.read_csv('train_data.csv', header = 0, sep = 't')
  df_train = df_train.dropna(subset=['tweettext_proc'])

  labels_dict = {'Not informative': 0, 'Related and informative':1}
  df_train['labels'] = df_train.cat_informativeness.replace(labels_dict)

  X_train = df_train['tweettext_proc']
  y_train = tf.keras.utils.to_categorical(df_train['labels'], num_classes=2)

  return X_train, y_train

In [ ]:
def get_test_data(category):
  df_test = pd.read_csv('test_data.csv', header = 0, sep = 't')
  df_test = df_test.dropna(subset=['tweettext_proc'])

  labels_dict = {'Not informative': 0, 'Related and informative':1}
  df_test['labels'] = df_test.cat_informativeness.replace(labels_dict)

  test = df_test.loc[df_test['categorization_type'] == category]

  X_test = test['tweettext_proc']
  y_test = tf.keras.utils.to_categorical(test['labels'], num_classes=2)

  return X_test, y_test

### Running the model
In scenario 2 the model is trained using data from all categories and tested on data for three different categories separately. The same create_model() and evaluate() function as in scenario 1 are used.

In [ ]:
def run_scenario2(learning_rate=1e-3, dropout=0.1, epochs=5, batch_size=16): 

  # Get training data
  X_train, y_train = get_train_data()

  # Training
  bert_model = create_bert(learning_rate, dropout)
  hist = bert_model.fit(x=X_train, y=y_train, epochs=epochs, batch_size=batch_size, validation_split=.15)

  for cat in ['Earthquake', 'Floods', 'Derailment']:

      # Get test data
      X_test, y_test = get_test_data(cat)

      #Testing
      y_test_arg = np.argmax(y_test, axis=1)
      y_pred = np.argmax(bert_model.predict(X_test),axis=1)

      print(cat)
      accuracy, precision, recall, f1score, mauc = evaluate(y_test_arg, y_pred, 0.5)

In [ ]:
run_scenario2()

Epoch 1/5
1185/1185 [==============================] - 704s 580ms/step - loss: 0.6520 - accuracy: 0.6362 - val_loss: 0.6487 - val_accuracy: 0.5972
Epoch 2/5
1185/1185 [==============================] - 688s 581ms/step - loss: 0.6039 - accuracy: 0.6789 - val_loss: 0.5921 - val_accuracy: 0.6890
Epoch 3/5
1185/1185 [==============================] - 642s 542ms/step - loss: 0.5780 - accuracy: 0.6952 - val_loss: 0.5942 - val_accuracy: 0.6678
Epoch 4/5
1185/1185 [==============================] - 688s 581ms/step - loss: 0.5671 - accuracy: 0.7061 - val_loss: 0.6723 - val_accuracy: 0.6103
Epoch 5/5
32/32 [==============================] - 31s 940ms/step
Earthquake
                 precision    recall  f1-score   support

Not informative       0.64      0.78      0.70       373
    Informative       0.85      0.74      0.79       627

       accuracy                           0.75      1000
      macro avg       0.74      0.76      0.75      1000
   weighted avg       0.77      0.75      0.76  

### Hyperparameter tuning for scenario 2
The best hyperparameters are found by randomly choosing a combination of values for the following hyperparameters:

* Learning Rate using log scale
* Number of epochs
* Batch size
* Dropout

In [ ]:
rng = np.random.RandomState(4)
param_dist = {    'dropout': [0.1, 0.3],
                  'learning_rate': loguniform.rvs(1e-5, 1e-2, size= 10),
                  'epochs': [2, 5, 10, 25],
                  'batch_size': [16, 32, 64, 128]
                  }
dict_parameters = ParameterSampler(param_distributions=param_dist, n_iter=10, random_state=rng)

In [ ]:
for parameters in dict_parameters:
    print("===============Now testing the following parameters===============")
    print("Learning rate: ", parameters['learning_rate'])
    print("Dropout: ", parameters['dropout'])
    print("Epochs: ", parameters['epochs'])
    print("Batch size: ", parameters['batch_size'])
    print()

    run_scenario2(learning_rate=parameters['learning_rate'], dropout=parameters['dropout'], epochs=parameters['epochs'], batch_size=parameters['batch_size'])